**Requirements:**

Input parameters to be taken into account for pricing:

1.   Injection dates (inj_dates).
2.   Withdrawal dates (withd_dates).
3.   The prices at which the commodity can be purchased/sold.
4.   The rate at which the gas can be injected/withdrawn.
5.   The maximum volume that can be stored.
6.   Storage costs.


Write a function that takes these inputs and gives back the value of the contract. You can assume there is **no transport delay** and that **interest rates are zero**. Test your code by selecting a few sample inputs.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nat_gas_df = pd.read_csv("/content/Nat_Gas.csv", index_col=0)
nat_gas_df.index = pd.to_datetime(nat_gas_df.index)
nat_gas_df.tail(18)

<ipython-input-1-a7b7e9ab25ae>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  nat_gas_df.index = pd.to_datetime(nat_gas_df.index)


,Prices
Dates,
2023-04-30,11.5
2023-05-31,11.2
2023-06-30,10.9
2023-07-31,11.4
2023-08-31,11.1
2023-09-30,11.5
2023-10-31,11.8
2023-11-30,12.2
2023-12-31,12.8


In [2]:
inj_dates = ['2023-06-30','2023-07-31','2023-08-31']
inj_prices = nat_gas_df.loc[inj_dates]['Prices']
inj_prices

,Prices
Dates,
2023-06-30,10.9
2023-07-31,11.4
2023-08-31,11.1


In [3]:
withd_dates = ['2023-11-30','2023-12-31','2023-01-31']
withd_prices = nat_gas_df.loc[withd_dates]['Prices']
withd_prices

,Prices
Dates,
2023-11-30,12.2
2023-12-31,12.8
2023-01-31,12.1


In [4]:
def pricing_model(inj_dates,
                  withd_dates,
                  prices,
                  rate,
                  maximum_volume,
                  storage_costs,
                  inj_cost_per_unit=0,
                  withd_cost_per_unit=0,
                  transport_costs=0):

    total_inj_volume = len(inj_dates) * rate
    total_withd_volume = len(withd_dates) * rate

    if total_inj_volume > maximum_volume:
        return 'Error: Total injected volume exceeds maximum storable volume.'

    # injection costs (purchase + per-unit injection + transport)
    inj_costs = sum(prices.loc[date] * rate for date in inj_dates)  # the total amount paid to buy the gas at the injection dates.
    inj_additional_costs = (inj_cost_per_unit * rate + transport_costs) * len(inj_dates) # operational costs.

    # withdrawal revenues (sale - per-unit withdrawal - transport)
    withd_revenues = sum(prices.loc[date] * rate for date in withd_dates)  # the total revenue from selling the gas on the withdrawal dates.
    withd_deductions = (withd_cost_per_unit * rate + transport_costs) * len(withd_dates)

    total_storage_costs = storage_costs * len(inj_dates)
    net_value = (withd_revenues - withd_deductions) - (inj_costs + inj_additional_costs + total_storage_costs)

    return net_value


rate = 1_000_000                    # volume inject/withdraw
maximum_volume = (3*1_000_000)      # maximum volume that can be stored
storage_costs = 100_000             # per month
inj_cost_per_unit = 0.01            # $0.01 per MMBtu
withd_cost_per_unit = 0.01          # $0.01 per MMBtu
transport_costs = 50_000            # per trip

net_value = pricing_model(
    inj_dates = inj_dates,
    withd_dates = withd_dates,
    prices = nat_gas_df['Prices'],
    rate = rate,
    maximum_volume = maximum_volume,
    storage_costs = storage_costs,
    inj_cost_per_unit = inj_cost_per_unit,
    withd_cost_per_unit = withd_cost_per_unit,
    transport_costs = transport_costs
)

if isinstance(net_value, str):
    print(net_value)
else:
    print(f"Estimated contract value: ${net_value:,.0f}")


Estimated contract value: $3,040,000
